In [1]:
import pandas as pd
import evaluate
from calculate_tsed import *
import code_bert_score


In [2]:
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

In [3]:
df_1500_sample = pd.read_csv('random_1500_stratified_samples.csv')
df_1500_sample.head()

,id,repository_name,file_path,class_name,human_written_code,class_skeleton,total_program_units,total_doc_str,snippet_id,coverage,gpt-o4-mini_relevant,qwen-3-coder_relevant,claude-4-sonnet_relevant,docstr_type
0,740197,sirrice/pygg,sirrice_pygg/pygg/pygg.py,pygg.pygg.GGData,"class GGData(object):\n def __init__(self, r_...",class GGData(object):\n\n\n def __init__(self...,3,0,snippet_368,1.0,class GGData(object):\n\n def __init__(self...,"class GGData(object):\n def __init__(self, ...",class GGData(object):\n\n def __init__(self...,no_docstr
1,361060,eerimoq/textparser,/Users/umroot/Documents/PhD_works/PhD-Core-Con...,errors.parsita_json,class parsita_json(object):\n\n @staticmeth...,class parsita_json(object):\n\n\n @staticme...,3,0,snippet_216,1.0,class parsita_json(object):\n\n @staticmeth...,class parsita_json(object):\n @staticmethod...,class parsita_json(object):\n\n @staticmeth...,no_docstr
2,830433,what-studio/profiling,what-studio_profiling/profiling/__main__.py,profiling.__main__.Params,class Params(object):\n\n def __init__(self...,class Params(object):\n\n\n def __init__(se...,4,0,snippet_505,1.0,"class Params(object):\n def __init__(self, ...","class Params(object):\n def __init__(self, ...","class Params(object):\n def __init__(self, ...",no_docstr
3,267469,blazelibs/blazeutils,blazelibs_blazeutils/blazeutils/strings.py,blazeutils.strings.StringIndenter,class StringIndenter(object):\n\n def __ini...,class StringIndenter(object):\n\n\n def __i...,8,0,snippet_190,1.0,class StringIndenter(object):\n def __init_...,class StringIndenter(object):\n def __init_...,class StringIndenter(object):\n\n def __ini...,no_docstr
4,352284,drestebon/papageorge,drestebon_papageorge/lib/papageorge/model.py,papageorge.model.BB,"class BB(object):\n def __init__(self, stat...","class BB(object):\n\n\n def __init__(self, ...",3,0,snippet_501,1.0,"class BB(object):\n\n def __init__(self, st...","class BB(object):\n def __init__(self, stat...","class BB(object):\n\n def __init__(self, st...",no_docstr


In [4]:
df_1500_sample.shape

(1500, 14)

In [5]:
df_1500_sample.columns

Index(['id', 'repository_name', 'file_path', 'class_name',
       'human_written_code', 'class_skeleton', 'total_program_units',
       'total_doc_str', 'snippet_id', 'coverage', 'gpt-o4-mini_relevant',
       'qwen-3-coder_relevant', 'claude-4-sonnet_relevant', 'docstr_type'],
      dtype='object')

In [6]:
df_1500_sample.docstr_type.value_counts(normalize=True)

docstr_type
no_docstr         0.333333
partial_docstr    0.333333
full_docstr       0.333333
Name: proportion, dtype: float64

In [7]:
def calculate_metrics(references, predictions, model_name):
    print(f"Calculating metrics for {model_name}")
    rouge_scores = rouge.compute(predictions = predictions,
                          references = references, use_aggregator = False, 
                          rouge_types = ['rouge1', 'rouge2', 'rouge3', 'rougeL'])
    
    rouge1_value = rouge_scores['rouge1']
    rouge2_value = rouge_scores['rouge2']
    rouge3_value = rouge_scores['rouge3']
    rougeL_value = rouge_scores['rougeL']

    bleu_value = []
    bleu1_value = []
    bleu2_value = []
    bleu3_value = []
    for i in range(len(predictions)):
        try:
            bleu_score = bleu.compute(predictions=[predictions[i]], references=[references[i]], max_order = 3,
                                    smooth = True)
            bleu_value.append(bleu_score['bleu'])
            bleu1_value.append(bleu_score['precisions'][0])
            bleu2_value.append(bleu_score['precisions'][1])
            bleu3_value.append(bleu_score['precisions'][2])
        except:
            bleu_value.append(None)
            bleu1_value.append(None)
            bleu2_value.append(None)
            bleu3_value.append(None)
    
    code_bert_prec = []
    code_bert_recall = []
    code_bert_f1 = []
    code_bert_f3 = []
    for i in range(len(predictions)):
        try:
            code_bert_metric = code_bert_score.score(cands=[predictions[i]], refs=[references[i]], lang='python')
            code_bert_prec.append(float(code_bert_metric[0][0]))
            code_bert_recall.append(float(code_bert_metric[1][0]))
            code_bert_f1.append(float(code_bert_metric[2][0]))
            code_bert_f3.append(float(code_bert_metric[3][0]))
        except:
            code_bert_prec.append(None)
            code_bert_recall.append(None)
            code_bert_f1.append(None)
            code_bert_f3.append(None)
    
    tsed_values = []
    for i in range(len(predictions)):
        try:
            tsed = Calculate('python', references[i], predictions[i], .33333, .33333, .33333)    
            tsed_values.append(tsed)
        except Exception as e:
            print(f"Error calculating TS-ED for index {i}: {e}")
            tsed_values.append(None)
    
    metrics_df = pd.DataFrame(
        {
            f'rouge1_{model_name}': rouge1_value,
            f'rouge2_{model_name}': rouge2_value,
            f'rouge3_{model_name}': rouge3_value,
            f'rougeL_{model_name}': rougeL_value,
            f'bleu_{model_name}': bleu_value,
            f'bleu1_{model_name}': bleu1_value,
            f'bleu2_{model_name}': bleu2_value,
            f'bleu3_{model_name}': bleu3_value,
            f'code_bert_prec_{model_name}': code_bert_prec,
            f'code_bert_recall_{model_name}': code_bert_recall,
            f'code_bert_f1_{model_name}': code_bert_f1,
            f'code_bert_f3_{model_name}': code_bert_f3,
            f'tsed_{model_name}': tsed_values
        }
        )
    return metrics_df

In [8]:
gpt_metrics = calculate_metrics(references=df_1500_sample['human_written_code'].tolist(),
                  predictions=df_1500_sample['gpt-o4-mini_relevant'].tolist(),
                  model_name='gpt-o4-mini')

qwen_metrics = calculate_metrics(references=df_1500_sample['human_written_code'].tolist(),
                  predictions=df_1500_sample['qwen-3-coder_relevant'].tolist(),
                  model_name='qwen-3-coder')

claude_metrics = calculate_metrics(references=df_1500_sample['human_written_code'].tolist(),
                  predictions=df_1500_sample['claude-4-sonnet_relevant'].tolist(),
                  model_name='claude-4-sonnet')

final_metrics_df = pd.concat([df_1500_sample.reset_index(drop=True), 
                              gpt_metrics, 
                              qwen_metrics, 
                              claude_metrics], axis=1)

Calculating metrics for gpt-o4-mini


/opt/anaconda3/envs/OpenClassGen/lib/python3.11/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


Error calculating TS-ED for index 80: encoding without a string argument
Error calculating TS-ED for index 134: encoding without a string argument
Error calculating TS-ED for index 299: encoding without a string argument
Error calculating TS-ED for index 322: encoding without a string argument
Error calculating TS-ED for index 414: encoding without a string argument
Error calculating TS-ED for index 420: encoding without a string argument
Error calculating TS-ED for index 496: encoding without a string argument
Error calculating TS-ED for index 504: encoding without a string argument
Error calculating TS-ED for index 511: encoding without a string argument
Error calculating TS-ED for index 629: encoding without a string argument
Error calculating TS-ED for index 687: encoding without a string argument
Error calculating TS-ED for index 818: encoding without a string argument
Error calculating TS-ED for index 867: encoding without a string argument
Error calculating TS-ED for index 897: 

In [9]:
final_metrics_df.head(3)

,id,repository_name,file_path,class_name,human_written_code,class_skeleton,total_program_units,total_doc_str,snippet_id,coverage,...,rougeL_claude-4-sonnet,bleu_claude-4-sonnet,bleu1_claude-4-sonnet,bleu2_claude-4-sonnet,bleu3_claude-4-sonnet,code_bert_prec_claude-4-sonnet,code_bert_recall_claude-4-sonnet,code_bert_f1_claude-4-sonnet,code_bert_f3_claude-4-sonnet,tsed_claude-4-sonnet
0,740197,sirrice/pygg,sirrice_pygg/pygg/pygg.py,pygg.pygg.GGData,"class GGData(object):\n def __init__(self, r_...",class GGData(object):\n\n\n def __init__(self...,3,0,snippet_368,1.0,...,0.704225,0.501218,0.518868,0.504762,0.480769,0.894179,0.945959,0.919340,0.940512,0.854168
1,361060,eerimoq/textparser,/Users/umroot/Documents/PhD_works/PhD-Core-Con...,errors.parsita_json,class parsita_json(object):\n\n @staticmeth...,class parsita_json(object):\n\n\n @staticme...,3,0,snippet_216,1.0,...,0.545455,0.515782,0.564103,0.500000,0.486486,0.853202,0.898200,0.875123,0.893488,0.822224
2,830433,what-studio/profiling,what-studio_profiling/profiling/__main__.py,profiling.__main__.Params,class Params(object):\n\n def __init__(self...,class Params(object):\n\n\n def __init__(se...,4,0,snippet_505,1.0,...,0.467290,0.341003,0.439024,0.325153,0.277778,0.720436,0.843515,0.777132,0.829346,0.782948


In [10]:
code_bleu_metrics_df = pd.read_csv('1500_samples_codebleu_metrics.csv')
final_metrics_df = pd.concat([final_metrics_df, code_bleu_metrics_df], axis=1)
final_metrics_df.head(3)

,id,repository_name,file_path,class_name,human_written_code,class_skeleton,total_program_units,total_doc_str,snippet_id,coverage,...,qwen-3-coder_code_bleu,qwen-3-coder_ngram_match,qwen-3-coder_weighted_ngram_match,qwen-3-coder_syntax_match,qwen-3-coder_dataflow_match,claude-4-sonnet_code_bleu,claude-4-sonnet_ngram_match,claude-4-sonnet_weighted_ngram_match,claude-4-sonnet_syntax_match,claude-4-sonnet_dataflow_match
0,740197,sirrice/pygg,sirrice_pygg/pygg/pygg.py,pygg.pygg.GGData,"class GGData(object):\n def __init__(self, r_...",class GGData(object):\n\n\n def __init__(self...,3,0,snippet_368,1.0,...,0.775760,0.618793,0.883368,0.684211,0.916667,0.768544,0.589932,0.883368,0.684211,0.916667
1,361060,eerimoq/textparser,/Users/umroot/Documents/PhD_works/PhD-Core-Con...,errors.parsita_json,class parsita_json(object):\n\n @staticmeth...,class parsita_json(object):\n\n\n @staticme...,3,0,snippet_216,1.0,...,0.547391,0.007864,0.566316,0.615385,0.000000,0.517465,0.317023,0.522067,0.230769,0.000000
2,830433,what-studio/profiling,what-studio_profiling/profiling/__main__.py,profiling.__main__.Params,class Params(object):\n\n def __init__(self...,class Params(object):\n\n\n def __init__(se...,4,0,snippet_505,1.0,...,0.459507,0.359040,0.454747,0.424242,0.600000,0.342972,0.138474,0.250081,0.333333,0.650000


In [11]:
final_metrics_df.to_csv('model_comparison_metrics.csv', index=False)
final_metrics_df.shape
final_metrics_df.columns

Index(['id', 'repository_name', 'file_path', 'class_name',
       'human_written_code', 'class_skeleton', 'total_program_units',
       'total_doc_str', 'snippet_id', 'coverage', 'gpt-o4-mini_relevant',
       'qwen-3-coder_relevant', 'claude-4-sonnet_relevant', 'docstr_type',
       'rouge1_gpt-o4-mini', 'rouge2_gpt-o4-mini', 'rouge3_gpt-o4-mini',
       'rougeL_gpt-o4-mini', 'bleu_gpt-o4-mini', 'bleu1_gpt-o4-mini',
       'bleu2_gpt-o4-mini', 'bleu3_gpt-o4-mini', 'code_bert_prec_gpt-o4-mini',
       'code_bert_recall_gpt-o4-mini', 'code_bert_f1_gpt-o4-mini',
       'code_bert_f3_gpt-o4-mini', 'tsed_gpt-o4-mini', 'rouge1_qwen-3-coder',
       'rouge2_qwen-3-coder', 'rouge3_qwen-3-coder', 'rougeL_qwen-3-coder',
       'bleu_qwen-3-coder', 'bleu1_qwen-3-coder', 'bleu2_qwen-3-coder',
       'bleu3_qwen-3-coder', 'code_bert_prec_qwen-3-coder',
       'code_bert_recall_qwen-3-coder', 'code_bert_f1_qwen-3-coder',
       'code_bert_f3_qwen-3-coder', 'tsed_qwen-3-coder',
       'rouge1_claude

In [12]:
#final_metrics_df.info()
final_metrics_df.isna().sum()

id                                      0
repository_name                         0
file_path                               0
class_name                              0
human_written_code                      0
                                       ..
claude-4-sonnet_code_bleu               2
claude-4-sonnet_ngram_match             2
claude-4-sonnet_weighted_ngram_match    2
claude-4-sonnet_syntax_match            2
claude-4-sonnet_dataflow_match          2
Length: 68, dtype: int64

In [13]:
final_metrics_df.dropna().shape

(1471, 68)

In [14]:
final_metrics_df.dropna(inplace=True)
final_metrics_df.shape

(1471, 68)

In [15]:
final_metrics_df.to_csv('model_comparison_metrics_no_na.csv', index=False)

In [16]:
final_metrics_df.iloc[:, 14:].describe()

,rouge1_gpt-o4-mini,rouge2_gpt-o4-mini,rouge3_gpt-o4-mini,rougeL_gpt-o4-mini,bleu_gpt-o4-mini,bleu1_gpt-o4-mini,bleu2_gpt-o4-mini,bleu3_gpt-o4-mini,code_bert_prec_gpt-o4-mini,code_bert_recall_gpt-o4-mini,...,qwen-3-coder_code_bleu,qwen-3-coder_ngram_match,qwen-3-coder_weighted_ngram_match,qwen-3-coder_syntax_match,qwen-3-coder_dataflow_match,claude-4-sonnet_code_bleu,claude-4-sonnet_ngram_match,claude-4-sonnet_weighted_ngram_match,claude-4-sonnet_syntax_match,claude-4-sonnet_dataflow_match
count,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,...,1471.000000,1.471000e+03,1471.000000,1471.000000,1471.000000,1471.000000,1.471000e+03,1471.000000,1471.000000,1471.000000
mean,0.664793,0.535334,0.465524,0.603756,0.481586,0.690344,0.543554,0.470242,0.867198,0.881397,...,0.481295,4.096110e-01,0.484709,0.489258,0.491296,0.493628,4.187043e-01,0.486995,0.505277,0.502352
std,0.183889,0.219839,0.230646,0.202191,0.188360,0.190247,0.197783,0.197439,0.079300,0.070786,...,0.175345,2.308715e-01,0.225627,0.178607,0.267209,0.171801,2.344178e-01,0.224493,0.169749,0.269478
min,0.088435,0.034247,0.013793,0.081633,0.000006,0.082969,0.062162,0.027100,0.587904,0.546069,...,0.042045,7.734309e-09,0.009158,0.010526,0.000000,0.042045,3.575572e-07,0.007449,0.045455,0.000000
25%,0.534564,0.363336,0.272727,0.445684,0.345687,0.570595,0.398383,0.316857,0.820082,0.835153,...,0.360210,2.155340e-01,0.306829,0.371281,0.294118,0.374879,2.118481e-01,0.303877,0.389831,0.314006
50%,0.685714,0.542857,0.464000,0.614650,0.491914,0.724884,0.552239,0.466667,0.878950,0.892454,...,0.487472,4.287233e-01,0.517341,0.474359,0.483871,0.498498,4.379876e-01,0.522154,0.489796,0.500000
75%,0.807925,0.708295,0.643073,0.762632,0.620139,0.840745,0.698569,0.621132,0.926962,0.935776,...,0.592210,5.826589e-01,0.658823,0.592593,0.674457,0.610547,6.037785e-01,0.662481,0.597694,0.680786
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000


In [17]:
final_metrics_df.docstr_type.value_counts(normalize=False)

docstr_type
no_docstr         492
full_docstr       491
partial_docstr    488
Name: count, dtype: int64

In [18]:
final_500 = pd.concat([final_metrics_df[final_metrics_df.docstr_type == 'full_docstr'].sample(167, random_state=42),
                       final_metrics_df[final_metrics_df.docstr_type == 'no_docstr'].sample(167, random_state=42),
                       final_metrics_df[final_metrics_df.docstr_type == 'partial_docstr'].sample(166, random_state=42)],
                      axis=0)
final_500.shape

(500, 68)

In [19]:
final_500.to_csv('model_comparison_metrics_500_balanced.csv', index=False)

In [20]:
for _, row in final_500.iterrows():
    f = open(f"functional_correctness/generated_classes/GPT-o4/{row['docstr_type']}/{row['snippet_id']}.py", "w")
    f.write(row['gpt-o4-mini_relevant'])
    f.close()
    f = open(f"functional_correctness/generated_classes/Qwen3-coder/{row['docstr_type']}/{row['snippet_id']}.py", "w")
    f.write(row['qwen-3-coder_relevant'])
    f.close()
    f = open(f"functional_correctness/generated_classes/Claude4-sonnet/{row['docstr_type']}/{row['snippet_id']}.py", "w")
    f.write(row['claude-4-sonnet_relevant'])
    f.close()
final_500.to_csv('final_500_model_comparison_metrics_no_na.csv', index=False)

In [33]:
# for _, row in final_500.iterrows():
#     f_read = open(f"../PhD-Core-Contents/Class-level-dataset-curation/data/metadata_folder/test_metadata/pynguin_generated_test_cases/{row['docstr_type']}/dynamosa_tests/test_{row['snippet_id']}.py", "r")
#     test_code = f_read.read()
#     f_read.close()
#     f_write_ind = open(f"functional_correctness/pynguin_generated_test_cases/{row['docstr_type']}_classes/test_{row['snippet_id']}.py", "w")
#     f_write_ind.write(test_code)
#     f_write_ind.close()
#     f_write_comb = open(f"functional_correctness/pynguin_generated_test_cases/all_combined/test_{row['docstr_type']}_{row['snippet_id']}.py", "w")
#     f_write_comb.write(test_code)
#     f_write_comb.close()